In [1]:
import sys
import os
from eegraphsc import *

In [21]:
import csv
import tarfile
import sys
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import scipy.io
from mne import Epochs, find_events
import mne
from mne_connectivity import spectral_connectivity_epochs
import glob
#import eegraph

from mne import find_events
#os.chdir('../..')
os.getcwd()
#sys.path.append('c:/users/pesylvestre/appdata/roaming/python/python310/site-packages')

'C:\\Users\\pesylvestre\\Documents\\Informatique\\BRAIL\\FY2023 In-House PoC3'

In [14]:
def eeg_to_epoch(path, event_id=dict(T=1), tmin=-0.2, tmax=0.8, picks='eeg'):
    #need a .vhdr
    
    with open(path, 'r') as file:
        filedata = file.read()
    filedata = filedata.replace(filedata[filedata.find("DataFile=")+len("DataFile="):
                                         filedata.find(".eeg")+1],
                                path.split("/")[-1].replace("vhdr", ""))
    with open(path, 'w') as file:
        file.write(filedata)
    raw = mne.io.read_raw_brainvision(path)
    #->
    raw.drop_channels("VEOG")
    montage = mne.channels.make_standard_montage("easycap-M1")
    print(montage)
    montage.plot()
    raw.set_montage(montage)
    #->
    raw.plot()
    raw.load_data()
    filt_raw = raw.copy().filter(0.1, 30, picks='eeg', method='iir', 
                             iir_params=dict(order=8, ftype='butter'), 
                             verbose=True)
    events = mne.events_from_annotations(filt_raw)
    epochs = mne.Epochs(filt_raw, events[0], event_id, tmin, tmax, 
                    picks=picks, baseline=(-0.2, 0), reject=None, preload=True)
    return epochs, events[0], events[1], raw, filt_raw

In [15]:
def tar_to_epoch(path, event_id=dict(T=1), tmin=-0.2, tmax=0.8, picks='eeg'):
    print("->",os.getcwd())
    tar = tarfile.open(path)
    d = tar.getnames()
    #print('/'.join(path.split("/")[0:-1]))
    tar.extractall(path='/'.join(path.split("/")[0:-1]))
    tar.close()
    print(d)
    return eeg_to_epoch(path.replace(d[0]+".tar.gz", '')+"\n".join(s for s in d if "vhdr" in s), event_id, tmin, tmax, picks), d[0]

In [5]:
def tar_to_data(path, event_id, dest):
    tar = tarfile.open(path)
    d = tar.getnames()
    print("->",path)
    temp = '/'.join(path.split("/")[0:-1])
    #print("<>",temp)
    tar.extractall(path='/'.join(path.split("/")[0:-1]))
    tar.close()
    with open("./In-HousePOC3/in_housepoc3/data/"+d[0]+"/RSEEG/"+d[0]+".vhdr", 'r') as file:
        data = file.readlines()
    data[5] = "DataFile="+d[0]+".eeg\n"
    data[6] = "MarkerFile="+d[0]+".vmrk\n"
    with open("./In-HousePOC3/in_housepoc3/data/"+d[0]+"/RSEEG/"+d[0]+".vhdr", 'w') as file:
        file.writelines(data)
    G = Graph()
    print("./In-HousePOC3/in_housepoc3/data/"+d[0]+"/RSEEG/"+d[0]+".vhdr")
    G.load_data(path="./In-HousePOC3/in_housepoc3/data/"+d[0]+"/RSEEG/"+d[0]+".vhdr")
    graphs, connectivity_matrix = G.modelate(window_size = 2, connectivity = 'pearson_correlation')
    n = len(graphs)
    res = []
    for i in range(0,n):
        temp = nx.clustering(graphs[i])
        try : 
            del temp['VEOG']
        except:
            print("VEOG")
        res += [list(temp.values())]


    with open(dest+d[0]+'.csv', 'w', newline='') as f:
        writer = csv.writer(f)
        writer.writerows(res)

In [ ]:
from glob import glob
sub_list = glob("C:/Users/pesylvestre/Documents/Informatique/BRAIL/FY2023 In-House PoC3/In-HousePOC3/in_housepoc3/data/*tar.gz")
for sub in sub_list:
    tar_to_data(sub.replace("\\", "/"), dict(eyesOpen=210, eyesClose=200), "C:/Users/pesylvestre/Documents/Informatique/BRAIL/FY2023 In-House PoC3/In-HousePOC3/in_housepoc3/pdata/")

-> C:/Users/pesylvestre/Documents/Informatique/BRAIL/FY2023 In-House PoC3/In-HousePOC3/in_housepoc3/data/sub-032301.tar.gz
./In-HousePOC3/in_housepoc3/data/sub-032301/RSEEG/sub-032301.vhdr
-> mne.io.read_raw_brainvision(self.path)
Extracting parameters from ./In-HousePOC3/in_housepoc3/data/sub-032301/RSEEG/sub-032301.vhdr...
Setting channel info structure...

EEG Information.
Number of Channels: 62
Sample rate: 2500.0 Hz.
Duration: 1022.0 seconds.
Channel Names: ['Fp1', 'Fp2', 'F7', 'F3', 'Fz', 'F4', 'F8', 'FC5', 'FC1', 'FC2', 'FC6', 'T7', 'C3', 'Cz', 'C4', 'T8', 'VEOG', 'CP5', 'CP1', 'CP2', 'CP6', 'AFz', 'P7', 'P3', 'Pz', 'P4', 'P8', 'PO9', 'O1', 'Oz', 'O2', 'PO10', 'AF7', 'AF3', 'AF4', 'AF8', 'F5', 'F1', 'F2', 'F6', 'FT7', 'FC3', 'FC4', 'FT8', 'C5', 'C1', 'C2', 'C6', 'TP7', 'CP3', 'CPz', 'CP4', 'TP8', 'P5', 'P1', 'P2', 'P6', 'PO7', 'PO3', 'POz', 'PO4', 'PO8']
Model Data.
Pearson_correlation_Estimator()
Intervals:  [(0, 5000.0), (5000.0, 10000.0), (10000.0, 15000.0), (15000.0, 20000.0


Threshold: 0.7
Number of graphs created: 511
-> C:/Users/pesylvestre/Documents/Informatique/BRAIL/FY2023 In-House PoC3/In-HousePOC3/in_housepoc3/data/sub-032302.tar.gz
./In-HousePOC3/in_housepoc3/data/sub-032302/RSEEG/sub-032302.vhdr
-> mne.io.read_raw_brainvision(self.path)
Extracting parameters from ./In-HousePOC3/in_housepoc3/data/sub-032302/RSEEG/sub-032302.vhdr...
Setting channel info structure...

EEG Information.
Number of Channels: 62
Sample rate: 2500.0 Hz.
Duration: 1007.94 seconds.
Channel Names: ['Fp1', 'Fp2', 'F7', 'F3', 'Fz', 'F4', 'F8', 'FC5', 'FC1', 'FC2', 'FC6', 'T7', 'C3', 'Cz', 'C4', 'T8', 'VEOG', 'CP5', 'CP1', 'CP2', 'CP6', 'AFz', 'P7', 'P3', 'Pz', 'P4', 'P8', 'PO9', 'O1', 'Oz', 'O2', 'PO10', 'AF7', 'AF3', 'AF4', 'AF8', 'F5', 'F1', 'F2', 'F6', 'FT7', 'FC3', 'FC4', 'FT8', 'C5', 'C1', 'C2', 'C6', 'TP7', 'CP3', 'CPz', 'CP4', 'TP8', 'P5', 'P1', 'P2', 'P6', 'PO7', 'PO3', 'POz', 'PO4', 'PO8']
Model Data.
Pearson_correlation_Estimator()
Intervals:  [(0, 5000.0), (5000.0, 


Threshold: 0.7
Number of graphs created: 504
-> C:/Users/pesylvestre/Documents/Informatique/BRAIL/FY2023 In-House PoC3/In-HousePOC3/in_housepoc3/data/sub-032303.tar.gz
./In-HousePOC3/in_housepoc3/data/sub-032303/RSEEG/sub-032303.vhdr
-> mne.io.read_raw_brainvision(self.path)
Extracting parameters from ./In-HousePOC3/in_housepoc3/data/sub-032303/RSEEG/sub-032303.vhdr...
Setting channel info structure...

EEG Information.
Number of Channels: 62
Sample rate: 2500.0 Hz.
Duration: 1022.04 seconds.
Channel Names: ['Fp1', 'Fp2', 'F7', 'F3', 'Fz', 'F4', 'F8', 'FC5', 'FC1', 'FC2', 'FC6', 'T7', 'C3', 'Cz', 'C4', 'T8', 'VEOG', 'CP5', 'CP1', 'CP2', 'CP6', 'AFz', 'P7', 'P3', 'Pz', 'P4', 'P8', 'PO9', 'O1', 'Oz', 'O2', 'PO10', 'AF7', 'AF3', 'AF4', 'AF8', 'F5', 'F1', 'F2', 'F6', 'FT7', 'FC3', 'FC4', 'FT8', 'C5', 'C1', 'C2', 'C6', 'TP7', 'CP3', 'CPz', 'CP4', 'TP8', 'P5', 'P1', 'P2', 'P6', 'PO7', 'PO3', 'POz', 'PO4', 'PO8']
Model Data.
Pearson_correlation_Estimator()
Intervals:  [(0, 5000.0), (5000.0, 


Threshold: 0.7
Number of graphs created: 512
-> C:/Users/pesylvestre/Documents/Informatique/BRAIL/FY2023 In-House PoC3/In-HousePOC3/in_housepoc3/data/sub-032304.tar.gz
./In-HousePOC3/in_housepoc3/data/sub-032304/RSEEG/sub-032304.vhdr
-> mne.io.read_raw_brainvision(self.path)
Extracting parameters from ./In-HousePOC3/in_housepoc3/data/sub-032304/RSEEG/sub-032304.vhdr...
Setting channel info structure...

EEG Information.
Number of Channels: 62
Sample rate: 2500.0 Hz.
Duration: 1096.56 seconds.
Channel Names: ['Fp1', 'Fp2', 'F7', 'F3', 'Fz', 'F4', 'F8', 'FC5', 'FC1', 'FC2', 'FC6', 'T7', 'C3', 'Cz', 'C4', 'T8', 'VEOG', 'CP5', 'CP1', 'CP2', 'CP6', 'AFz', 'P7', 'P3', 'Pz', 'P4', 'P8', 'PO9', 'O1', 'Oz', 'O2', 'PO10', 'AF7', 'AF3', 'AF4', 'AF8', 'F5', 'F1', 'F2', 'F6', 'FT7', 'FC3', 'FC4', 'FT8', 'C5', 'C1', 'C2', 'C6', 'TP7', 'CP3', 'CPz', 'CP4', 'TP8', 'P5', 'P1', 'P2', 'P6', 'PO7', 'PO3', 'POz', 'PO4', 'PO8']
Model Data.
Pearson_correlation_Estimator()
Intervals:  [(0, 5000.0), (5000.0, 

c:\users\pesylvestre\appdata\local\programs\python\python38\lib\site-packages\scipy\stats\_stats_py.py:4424: ConstantInputWarning: An input array is constant; the correlation coefficient is not defined.
  warnings.warn(stats.ConstantInputWarning(msg))


...


Threshold: 0.7
Number of graphs created: 504
-> C:/Users/pesylvestre/Documents/Informatique/BRAIL/FY2023 In-House PoC3/In-HousePOC3/in_housepoc3/data/sub-032314.tar.gz
./In-HousePOC3/in_housepoc3/data/sub-032314/RSEEG/sub-032314.vhdr
-> mne.io.read_raw_brainvision(self.path)
Extracting parameters from ./In-HousePOC3/in_housepoc3/data/sub-032314/RSEEG/sub-032314.vhdr...
Setting channel info structure...

EEG Information.
Number of Channels: 62
Sample rate: 2500.0 Hz.
Duration: 1011.06 seconds.
Channel Names: ['Fp1', 'Fp2', 'F7', 'F3', 'Fz', 'F4', 'F8', 'FC5', 'FC1', 'FC2', 'FC6', 'T7', 'C3', 'Cz', 'C4', 'T8', 'VEOG', 'CP5', 'CP1', 'CP2', 'CP6', 'AFz', 'P7', 'P3', 'Pz', 'P4', 'P8', 'PO9', 'O1', 'Oz', 'O2', 'PO10', 'AF7', 'AF3', 'AF4', 'AF8', 'F5', 'F1', 'F2', 'F6', 'FT7', 'FC3', 'FC4', 'FT8', 'C5', 'C1', 'C2', 'C6', 'TP7', 'CP3', 'CPz', 'CP4', 'TP8', 'P5', 'P1', 'P2', 'P6', 'PO7', 'PO3', 'POz', 'PO4', 'PO8']
Model Data.
Pearson_correlation_Estimator()
Intervals:  [(0, 5000.0), (5000.0, 


Threshold: 0.7
Number of graphs created: 506
-> C:/Users/pesylvestre/Documents/Informatique/BRAIL/FY2023 In-House PoC3/In-HousePOC3/in_housepoc3/data/sub-032315.tar.gz
./In-HousePOC3/in_housepoc3/data/sub-032315/RSEEG/sub-032315.vhdr
-> mne.io.read_raw_brainvision(self.path)
Extracting parameters from ./In-HousePOC3/in_housepoc3/data/sub-032315/RSEEG/sub-032315.vhdr...
Setting channel info structure...

EEG Information.
Number of Channels: 62
Sample rate: 2500.0 Hz.
Duration: 1002.1 seconds.
Channel Names: ['Fp1', 'Fp2', 'F7', 'F3', 'Fz', 'F4', 'F8', 'FC5', 'FC1', 'FC2', 'FC6', 'T7', 'C3', 'Cz', 'C4', 'T8', 'VEOG', 'CP5', 'CP1', 'CP2', 'CP6', 'AFz', 'P7', 'P3', 'Pz', 'P4', 'P8', 'PO9', 'O1', 'Oz', 'O2', 'PO10', 'AF7', 'AF3', 'AF4', 'AF8', 'F5', 'F1', 'F2', 'F6', 'FT7', 'FC3', 'FC4', 'FT8', 'C5', 'C1', 'C2', 'C6', 'TP7', 'CP3', 'CPz', 'CP4', 'TP8', 'P5', 'P1', 'P2', 'P6', 'PO7', 'PO3', 'POz', 'PO4', 'PO8']
Model Data.
Pearson_correlation_Estimator()
Intervals:  [(0, 5000.0), (5000.0, 1


Threshold: 0.7
Number of graphs created: 502
-> C:/Users/pesylvestre/Documents/Informatique/BRAIL/FY2023 In-House PoC3/In-HousePOC3/in_housepoc3/data/sub-032316.tar.gz
./In-HousePOC3/in_housepoc3/data/sub-032316/RSEEG/sub-032316.vhdr
-> mne.io.read_raw_brainvision(self.path)
Extracting parameters from ./In-HousePOC3/in_housepoc3/data/sub-032316/RSEEG/sub-032316.vhdr...
Setting channel info structure...

EEG Information.
Number of Channels: 62
Sample rate: 2500.0 Hz.
Duration: 1004.36 seconds.
Channel Names: ['Fp1', 'Fp2', 'F7', 'F3', 'Fz', 'F4', 'F8', 'FC5', 'FC1', 'FC2', 'FC6', 'T7', 'C3', 'Cz', 'C4', 'T8', 'VEOG', 'CP5', 'CP1', 'CP2', 'CP6', 'AFz', 'P7', 'P3', 'Pz', 'P4', 'P8', 'PO9', 'O1', 'Oz', 'O2', 'PO10', 'AF7', 'AF3', 'AF4', 'AF8', 'F5', 'F1', 'F2', 'F6', 'FT7', 'FC3', 'FC4', 'FT8', 'C5', 'C1', 'C2', 'C6', 'TP7', 'CP3', 'CPz', 'CP4', 'TP8', 'P5', 'P1', 'P2', 'P6', 'PO7', 'PO3', 'POz', 'PO4', 'PO8']
Model Data.
Pearson_correlation_Estimator()
Intervals:  [(0, 5000.0), (5000.0, 


Threshold: 0.7
Number of graphs created: 503
-> C:/Users/pesylvestre/Documents/Informatique/BRAIL/FY2023 In-House PoC3/In-HousePOC3/in_housepoc3/data/sub-032317.tar.gz
./In-HousePOC3/in_housepoc3/data/sub-032317/RSEEG/sub-032317.vhdr
-> mne.io.read_raw_brainvision(self.path)
Extracting parameters from ./In-HousePOC3/in_housepoc3/data/sub-032317/RSEEG/sub-032317.vhdr...
Setting channel info structure...

EEG Information.
Number of Channels: 62
Sample rate: 2500.0 Hz.
Duration: 1017.04 seconds.
Channel Names: ['Fp1', 'Fp2', 'F7', 'F3', 'Fz', 'F4', 'F8', 'FC5', 'FC1', 'FC2', 'FC6', 'T7', 'C3', 'Cz', 'C4', 'T8', 'VEOG', 'CP5', 'CP1', 'CP2', 'CP6', 'AFz', 'P7', 'P3', 'Pz', 'P4', 'P8', 'PO9', 'O1', 'Oz', 'O2', 'PO10', 'AF7', 'AF3', 'AF4', 'AF8', 'F5', 'F1', 'F2', 'F6', 'FT7', 'FC3', 'FC4', 'FT8', 'C5', 'C1', 'C2', 'C6', 'TP7', 'CP3', 'CPz', 'CP4', 'TP8', 'P5', 'P1', 'P2', 'P6', 'PO7', 'PO3', 'POz', 'PO4', 'PO8']
Model Data.
Pearson_correlation_Estimator()
Intervals:  [(0, 5000.0), (5000.0, 


Threshold: 0.7
Number of graphs created: 509
-> C:/Users/pesylvestre/Documents/Informatique/BRAIL/FY2023 In-House PoC3/In-HousePOC3/in_housepoc3/data/sub-032318.tar.gz
./In-HousePOC3/in_housepoc3/data/sub-032318/RSEEG/sub-032318.vhdr
-> mne.io.read_raw_brainvision(self.path)
Extracting parameters from ./In-HousePOC3/in_housepoc3/data/sub-032318/RSEEG/sub-032318.vhdr...
Setting channel info structure...

EEG Information.
Number of Channels: 62
Sample rate: 2500.0 Hz.
Duration: 1039.54 seconds.
Channel Names: ['Fp1', 'Fp2', 'F7', 'F3', 'Fz', 'F4', 'F8', 'FC5', 'FC1', 'FC2', 'FC6', 'T7', 'C3', 'Cz', 'C4', 'T8', 'VEOG', 'CP5', 'CP1', 'CP2', 'CP6', 'AFz', 'P7', 'P3', 'Pz', 'P4', 'P8', 'PO9', 'O1', 'Oz', 'O2', 'PO10', 'AF7', 'AF3', 'AF4', 'AF8', 'F5', 'F1', 'F2', 'F6', 'FT7', 'FC3', 'FC4', 'FT8', 'C5', 'C1', 'C2', 'C6', 'TP7', 'CP3', 'CPz', 'CP4', 'TP8', 'P5', 'P1', 'P2', 'P6', 'PO7', 'PO3', 'POz', 'PO4', 'PO8']
Model Data.
Pearson_correlation_Estimator()
Intervals:  [(0, 5000.0), (5000.0, 

In [ ]:
#os.getcwd()

In [ ]:
#Manualy download lemon data from : http://fcon_1000.projects.nitrc.org/indi/retro/MPI_LEMON/downloads/download_EEG.html
#epv = tar_to_epoch("./In-HousePOC3/in_housepoc3/data/sub-032301.tar.gz", event_id=dict(eyesOpen=210, eyesClose=200))

In [ ]:
res = spectral_connectivity_epochs(epv[0], fmin=9, fmax=11, faverage=True).get_data()[:, 0]
print(res)
res.shape

In [ ]:
G = Graph()
G.load_data(path="./In-HousePOC3/in_housepoc3/data/sub-032301/RSEEG/sub-032301.vhdr")

In [ ]:
graphs, connectivity_matrix = G.modelate(window_size = 2, connectivity = 'pearson_correlation')

In [ ]:
print(len(graphs))
print(len(graphs[0]))

In [ ]:
G.visualize_html(graphs[0], 'graph_1')

In [ ]:
#print(G.is_directed())
nx.is_directed(graphs[0])

In [ ]:
r = nx.clustering(graphs[0])

In [ ]:
rp = list(r.values())
print(rp)

In [ ]:
n = len(graphs)
res = np.zeros((n,61)) # Pre-allocate matrix
for i in range(0,n):
    temp = nx.clustering(graphs[i])
    try : 
        del temp['VEOG']
    except:
        print("VEOG")
    #print(len(list(temp.values())))
    res[i,:] =list(temp.values())
print(res.shape)

In [ ]:
res.tofile('C:/Users/pesylvestre/Documents/Informatique/BRAIL/FY2023 In-House PoC3/In-HousePOC3/in_housepoc3/pdata/'++'.csv', sep = ',')